<a href="https://colab.research.google.com/github/NewbieHocIT/soratori/blob/main/speech_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets librosa numpy hmmlearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from google.colab import drive

# Kết nối Google Drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `SoraToris` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticat

In [ ]:
pip install jiwer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 36.4 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import librosa
from hmmlearn import hmm
from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder

# Bước 1: Tải dataset từ Hugging Face
def load_common_voice(language="vi", split="train"):
    """
    Tải dataset Common Voice từ Hugging Face.
    """
    print("Đang tải dataset...")
    dataset = load_dataset("mozilla-foundation/common_voice_13_0", language, split=split)
    audio_files = []
    transcripts = []
    for entry in dataset:
        audio_files.append(entry["audio"]["path"])
        transcripts.append(entry["sentence"])
    return audio_files, transcripts

# Bước 2: Tạo bộ tokenizer (dựa trên ký tự)
class CharacterTokenizer:
    def __init__(self):
        self.encoder = None
        self.decoder = None

    def fit(self, sentences):
        """
        Tạo encoder và decoder từ danh sách các câu.
        """
        unique_chars = set("".join(sentences))  # Tập hợp các ký tự duy nhất
        self.encoder = {char: idx for idx, char in enumerate(sorted(unique_chars))}
        self.decoder = {idx: char for char, idx in self.encoder.items()}

    def encode(self, sentence):
        """
        Mã hóa một câu thành token ID.
        """
        return [self.encoder[char] for char in sentence]

    def decode(self, token_ids):
        """
        Giải mã token ID thành câu văn bản.
        """
        return "".join([self.decoder[token_id] for token_id in token_ids])

# Bước 3: Trích xuất đặc trưng MFCC từ âm thanh
def extract_features(file_path, n_mfcc=13):
    """
    Trích xuất đặc trưng MFCC từ một tệp âm thanh.
    """
    try:
        y, sr = librosa.load(file_path, sr=None)  # Tải tệp âm thanh
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)  # Trích xuất đặc trưng MFCC
        return mfcc.T  # Chuyển vị để phù hợp với HMM
    except Exception as e:
        print(f"Lỗi khi trích xuất đặc trưng từ {file_path}: {e}")
        return None

# Bước 4: Huấn luyện mô hình HMM
def train_hmm(features_list, n_components=5):
    """
    Huấn luyện mô hình HMM với các đặc trưng đã trích xuất.
    """
    try:
        all_features = np.concatenate(features_list, axis=0)
        model = hmm.GaussianHMM(n_components=n_components, covariance_type="diag", n_iter=100)
        model.fit(all_features)
        return model
    except Exception as e:
        print(f"Lỗi khi huấn luyện HMM: {e}")
        return None

# Bước 5: Chuẩn bị hệ thống Speech-to-Text
if __name__ == "__main__":
    # Đường dẫn tạm thời để kiểm tra
    os.environ["HF_HOME"] = "./huggingface"

    # Tải dữ liệu Common Voice
    language = "vi"  # Tiếng Việt
    audio_files, transcripts = load_common_voice(language=language, split="train")

    # Kiểm tra dữ liệu đã tải
    print(f"Tổng số mẫu dữ liệu: {len(audio_files)}")

    # Tạo bộ tokenizer
    tokenizer = CharacterTokenizer()
    tokenizer.fit(transcripts)

    # Kiểm tra tokenizer
    sample_sentence = transcripts[0]
    encoded = tokenizer.encode(sample_sentence)
    decoded = tokenizer.decode(encoded)
    print(f"Ví dụ token hóa:\n- Gốc: {sample_sentence}\n- Mã hóa: {encoded}\n- Giải mã: {decoded}")

    # Huấn luyện mô hình HMM cho từng câu
    models = {}
    for i, (audio_file, transcript) in enumerate(zip(audio_files[:20], transcripts[:20])):  # Chỉ dùng 10 tệp đầu tiên
        print(f"Đang xử lý tệp {i+1}/{10}: {audio_file}")
        features = extract_features(audio_file)
        if features is not None:
            model = train_hmm([features])
            if model:
                token_ids = tokenizer.encode(transcript)  # Mã hóa câu văn bản
                models[tuple(token_ids)] = model




Đang tải dataset...
Tổng số mẫu dữ liệu: 2462
Ví dụ token hóa:
- Gốc: quả nhiên trúng tuyển vào trường Quốc Lập
- Mã hóa: [45, 49, 86, 0, 42, 37, 38, 64, 42, 0, 48, 46, 72, 42, 36, 0, 48, 49, 52, 103, 42, 0, 50, 58, 43, 0, 48, 46, 83, 118, 42, 36, 0, 23, 49, 111, 33, 0, 19, 92, 44]
- Giải mã: quả nhiên trúng tuyển vào trường Quốc Lập
Đang xử lý tệp 1/10: /root/.cache/huggingface/datasets/downloads/extracted/58399062ad245dfaf8f114ef10011bc6da77257ce12add144a19ecc9c0bfcb6d/vi_train_0/common_voice_vi_23901117.mp3
Đang xử lý tệp 2/10: /root/.cache/huggingface/datasets/downloads/extracted/58399062ad245dfaf8f114ef10011bc6da77257ce12add144a19ecc9c0bfcb6d/vi_train_0/common_voice_vi_23901118.mp3
Đang xử lý tệp 3/10: /root/.cache/huggingface/datasets/downloads/extracted/58399062ad245dfaf8f114ef10011bc6da77257ce12add144a19ecc9c0bfcb6d/vi_train_0/common_voice_vi_23901119.mp3
Đang xử lý tệp 4/10: /root/.cache/huggingface/datasets/downloads/extracted/58399062ad245dfaf8f114ef10011bc6da77257ce12add144

In [ ]:
# Kiểm tra bộ tokenizer
print("Bộ mã hóa (Encoder):")
for char, idx in tokenizer.encoder.items():
    print(f"{char}: {idx}")

print("\nBộ giải mã (Decoder):")
for idx, char in tokenizer.decoder.items():
    print(f"{idx}: {char}")


Bộ mã hóa (Encoder):
 : 0
!: 1
": 2
': 3
,: 4
-: 5
.: 6
:: 7
;: 8
?: 9
A: 10
B: 11
C: 12
D: 13
E: 14
F: 15
G: 16
H: 17
K: 18
L: 19
M: 20
N: 21
P: 22
Q: 23
R: 24
S: 25
T: 26
U: 27
V: 28
X: 29
Y: 30
a: 31
b: 32
c: 33
d: 34
e: 35
g: 36
h: 37
i: 38
k: 39
l: 40
m: 41
n: 42
o: 43
p: 44
q: 45
r: 46
s: 47
t: 48
u: 49
v: 50
x: 51
y: 52
À: 53
Á: 54
Â: 55
Ô: 56
Ý: 57
à: 58
á: 59
â: 60
ã: 61
è: 62
é: 63
ê: 64
ì: 65
í: 66
ò: 67
ó: 68
ô: 69
õ: 70
ù: 71
ú: 72
ý: 73
Ă: 74
ă: 75
Đ: 76
đ: 77
ĩ: 78
ũ: 79
Ơ: 80
ơ: 81
Ư: 82
ư: 83
ạ: 84
Ả: 85
ả: 86
ấ: 87
ầ: 88
Ẩ: 89
ẩ: 90
ẫ: 91
ậ: 92
ắ: 93
ằ: 94
ẳ: 95
ẵ: 96
ặ: 97
ẹ: 98
ẻ: 99
ẽ: 100
ế: 101
ề: 102
ể: 103
ễ: 104
ệ: 105
ỉ: 106
ị: 107
ọ: 108
ỏ: 109
Ố: 110
ố: 111
Ồ: 112
ồ: 113
ổ: 114
ỗ: 115
ộ: 116
ớ: 117
ờ: 118
Ở: 119
ở: 120
ỡ: 121
ợ: 122
ụ: 123
ủ: 124
ứ: 125
Ừ: 126
ừ: 127
ử: 128
ữ: 129
ự: 130
ỳ: 131
ỵ: 132
ỷ: 133
ỹ: 134

Bộ giải mã (Decoder):
0:  
1: !
2: "
3: '
4: ,
5: -
6: .
7: :
8: ;
9: ?
10: A
11: B
12: C
13: D
14: E
15: F
16: G
17: H
18: K
19: L
20: M
21: N

In [ ]:
    # Thử nghiệm với một tệp mới
    test_file = audio_files[99]
    test_features = extract_features(test_file)

    # Dự đoán câu tương ứng
    if test_features is not None:
        best_score = float("-inf")
        best_transcript = None
        for token_ids, model in models.items():
            try:
                score = model.score(test_features)
                if score > best_score:
                    best_score = score
                    best_transcript = token_ids
            except Exception as e:
                print(f"Lỗi khi tính điểm với HMM: {e}")
                continue

        if best_transcript:
            predicted_sentence = tokenizer.decode(list(best_transcript))
            print(f"Dự đoán: {predicted_sentence}")
        else:
            print("Không tìm thấy câu phù hợp.")
    else:
        print("Không thể trích xuất đặc trưng từ tệp thử nghiệm.")

Dự đoán: Ông bà tao ngày ấy thì sợ lắm
